# Loading dataset and libraries

In [1]:
#Referencia
# https://www.kaggle.com/datasets/amirmotefaker/ab-testing-dataset

In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm # Chi²
from scipy.stats import ttest_ind # Test T

In [3]:
Control = pd.read_csv('/content/control_group.csv',delimiter = ';')
Variant = pd.read_csv('/content/test_group.csv',delimiter = ';')

In [4]:
columns = {
    "Campaign Name": "Campaign",
    "Date": "Date",
    "Spend [USD]": "Spent",
    "# of Impressions": "Impressions",
    "Reach": "Reach",
    "# of Website Clicks": "Clicks",
    "# of Searches": "Searches",
    "# of View Content": "Views",
    "# of Add to Cart": "Cartadd",
    "# of Purchase": "Purchases"
}

# Renomeando as colunas do control_data
Control.rename(columns=columns, inplace=True)

# Renomeando as colunas do test_data
Variant.rename(columns=columns, inplace=True)


# About the dataSet

## English (EN)

| Feature Name    | Description                                            |
|-----------------|--------------------------------------------------------|
| Campaign Name   | The name of the campaign                               |
| Date            | Date of the record                                      |
| Spent           | Amount spent on the campaign in dollars (How much you earn)                 |
| Impressions     | Number of impressions the ad crossed through the campaign |
| Reach           | The number of unique impressions received in the ad    |
| Clicks          | Number of website clicks received through the ads      |
| Searches        | Number of users who performed searches on the website  |
| Views           | Number of users who viewed content and products on the website |
| Cartadd         | Number of users who added products to the cart         |
| Purchases       | Number of purchases                                    |
| ARPPU     | average revenue per paying user ( Spent /    Purchases)                             |
| ARPOL      | Average Revenue Per Organic Lead ( Spent /    Impressions)            |




## Portugues (PT-BR)

| Nome da Característica | Descrição                                            |
|------------------------|-----------------------------------------------------|
| Nome da Campanha       | O nome da campanha                                  |
| Data                   | Data do registro                                     |
| Gasto                  | Quantia gasta na campanha em dólares (Quanto você ganha)            |
| Impressões             | Número de impressões que o anúncio teve durante a campanha |
| Alcance                | O número de impressões únicas recebidas no anúncio  |
| Cliques                | Número de cliques no site recebidos através dos anúncios |
| Pesquisas              | Número de usuários que realizaram pesquisas no site |
| Visualizações          | Número de usuários que visualizaram conteúdo e produtos no site |
| Adicionados ao Carrinho| Número de usuários que adicionaram produtos ao carrinho |
| Compras                | Número de compras                                   |
| ARPPU                  | Receita média por usuário pagante (Gasto / Compras) |
| ARPOL                  | Receita média por lead orgânico (Gasto / Impressões) |

# Data preprocessing

In [5]:
Control = Control.dropna()
Variant = Variant.dropna()

In [6]:
# Calcular ARPPU e ARPOL para o DataFrame Control
Control.loc[:, 'Arppu'] = Control['Spent'] / Control['Purchases']
Control.loc[:, 'Arpol'] = Control['Spent'] / Control['Impressions']

# Calcular ARPPU e ARPOL para o DataFrame Variant
Variant.loc[:, 'Arppu'] = Variant['Spent'] / Variant['Purchases']
Variant.loc[:, 'Arpol'] = Variant['Spent'] / Variant['Impressions']


In [ ]:
Control.describe()

In [ ]:
Variant.describe()

In [ ]:
Control.head(), Variant.head()

# Teste estatistico

## Teste Chi²

In [22]:
def Chi_quadrado(conv_a,obs_a,conv_b,obs_b):
  # Taxa de conversão para o Grupo A
  rate_a = conv_a / obs_a

  # Taxa de conversão para o Grupo B
  rate_b = conv_b / obs_b

  # Teste Z para duas proporções
  z_score, p_value = sm.stats.proportions_ztest([conv_a, conv_b], [obs_a, obs_b])

  # Intervalo de confiança para a diferença de proporções
  ci_low, ci_upp = sm.stats.proportion_confint([conv_a, conv_b], [obs_a, obs_b])

  print("Há relevancia?", p_value < 0.05)
  print("Taxa de conversão do Grupo A: {:.2f}%".format(rate_a))
  print("Taxa de conversão do Grupo B: {:.2f}%".format(rate_b))
  print("Teste Z score:", z_score)
  print("Valor p:", p_value)
  print("Intervalo de confiança para a diferença de proporções:", ci_low, ci_upp)
  if rate_a > rate_b:
    print("O Grupo A tem uma taxa de conversão maior.")
    print("A diferença percentual na taxa de conversão entre os grupos é de: {:.2f}%".format((rate_a - rate_b) * 100))
    print("A razão na taxa de conversão entre os grupos é de: {:.2f}%".format((rate_a / rate_b)))


  elif rate_b > rate_a:
    print("O Grupo B tem uma taxa de conversão maior.")
    print("A diferença percentual na taxa de conversão entre os grupos é de: {:.2f}%".format((rate_b - rate_a) * 100))
    print("A razão na taxa de conversão entre os grupos é de: {:.2f}%".format((rate_b / rate_a)))

  else:
    print("As taxas de conversão dos dois grupos são iguais.")

In [23]:
# Número de observações e conversões para o Grupo A e B
obs_a = Control['Impressions'].sum()
obs_b = Variant['Impressions'].sum()
conv_a = Control['Reach'].sum()
conv_b = Variant['Reach'].sum()

Chi_quadrado(conv_a,obs_a,conv_b,obs_b)

Há relevancia? True
Taxa de conversão do Grupo A: 0.81%
Taxa de conversão do Grupo B: 0.72%
Teste Z score: 256.080750326453
Valor p: 0.0
Intervalo de confiança para a diferença de proporções: [0.81049612 0.71660116] [0.81135723 0.71778136]
O Grupo A tem uma taxa de conversão maior.
A diferença percentual na taxa de conversão entre os grupos é de: 9.37%
A razão na taxa de conversão entre os grupos é de: 1.13%


In [24]:
# Número de observações e conversões para o Grupo A e B
obs_a = Control['Impressions'].sum()
obs_b = Variant['Impressions'].sum()
conv_a = Control['Purchases'].sum()
conv_b = Variant['Purchases'].sum()

Chi_quadrado(conv_a,obs_a,conv_b,obs_b)

Há relevancia? True
Taxa de conversão do Grupo A: 0.00%
Taxa de conversão do Grupo B: 0.01%
Teste Z score: -33.77499922226043
Valor p: 4.59330633997262e-250
Intervalo de confiança para a diferença de proporções: [0.00469599 0.00687931] [0.00484754 0.00709762]
O Grupo B tem uma taxa de conversão maior.
A diferença percentual na taxa de conversão entre os grupos é de: 0.22%
A razão na taxa de conversão entre os grupos é de: 1.46%


## Teste T

In [27]:
def Test_t(grupo_a,grupo_b):

  meana = grupo_a.mean()
  meanb = grupo_b.mean()

  # Executar o teste t de Student
  t_statistic, p_value = ttest_ind(grupo_a, grupo_b)

  # Imprimir os resultados
  print("Estatística t:", t_statistic)
  print("Valor p:", p_value)
  print("Média grupo A: {:.2f}".format(meana) )
  print("Média grupo A: {:.2f}".format(meanb) )
  print("Razão Grupos: {:.2f}%".format(meana / meanb - 1))

  # Interpretar os resultados
  if p_value < 0.05:
      print("Há uma diferença estatisticamente significativa entre as médias dos dois grupos.")
  else:
      print("Não há uma diferença estatisticamente significativa entre as médias dos dois grupos.")

In [28]:
grupo_a = Control['Arppu']
grupo_b = Variant['Arppu']

Test_t(grupo_a,grupo_b)

Estatística t: -1.307409392473657
Valor p: 0.19632297368244128
Média grupo A: 5.05
Média grupo A: 5.90
Razão Grupos: -0.14%
Não há uma diferença estatisticamente significativa entre as médias dos dois grupos.


In [29]:
grupo_a = Control['Arpol']
grupo_b = Variant['Arpol']

Test_t(grupo_a,grupo_b)

Estatística t: -4.663148680025627
Valor p: 1.9233082583529276e-05
Média grupo A: 0.02
Média grupo A: 0.04
Razão Grupos: -0.49%
Há uma diferença estatisticamente significativa entre as médias dos dois grupos.
